In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.tokenize import WordPunctTokenizer
import itertools
from pandas_profiling import ProfileReport
import numpy as np


In [80]:
df_train = pd.read_csv('data/train.csv')
df_train


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,[QUOTE=jessepinkman16;5a50ac34d89b093f368b456e...,kaskus,0,0,0,1
1,"@verosvante kita2 aja nitizen yang pada kepo,t...",instagram,0,0,0,0
2,"""#SidangAhok smg sipenista agama n ateknya mat...",twitter,0,1,1,1
3,@bolususulembang.jkt barusan baca undang2 ini....,instagram,0,0,0,0
4,bikin anak mulu lu nof \nkaga mikir apa kasian...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6990,Mas. Sebenarnya aku gamau bully mas tp kek nya...,instagram,0,0,0,0
6991,"Kdang ngaku nicky minaj, kdang beyonce, kdang ...",instagram,0,0,0,0
6992,"Time kemas sesuatu tadi, terfikir jugak botol ...",twitter,0,0,1,0
6993,"@BasherRL heh kontol, jan sok pemes lu di rp, ...",twitter,1,0,0,1


In [38]:
len(df_train)-len(df_train.drop_duplicates(subset=['text']))


624

In [39]:
df_train = df_train.drop_duplicates(ignore_index=True, subset=['text'])
df_train


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,[QUOTE=jessepinkman16;5a50ac34d89b093f368b456e...,kaskus,0,0,0,1
1,"@verosvante kita2 aja nitizen yang pada kepo,t...",instagram,0,0,0,0
2,"""#SidangAhok smg sipenista agama n ateknya mat...",twitter,0,1,1,1
3,@bolususulembang.jkt barusan baca undang2 ini....,instagram,0,0,0,0
4,bikin anak mulu lu nof \nkaga mikir apa kasian...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6366,Iya miris banget. Smua gara2 pengaruh barat ni...,kaskus,0,0,0,0
6367,"Kk luna, harus kuat ya! Harus sabar ngadepin s...",instagram,0,0,0,0
6368,Mas. Sebenarnya aku gamau bully mas tp kek nya...,instagram,0,0,0,0
6369,"Kdang ngaku nicky minaj, kdang beyonce, kdang ...",instagram,0,0,0,0


In [40]:
df_train['text']


0       [QUOTE=jessepinkman16;5a50ac34d89b093f368b456e...
1       @verosvante kita2 aja nitizen yang pada kepo,t...
2       "#SidangAhok smg sipenista agama n ateknya mat...
3       @bolususulembang.jkt barusan baca undang2 ini....
4       bikin anak mulu lu nof \nkaga mikir apa kasian...
                              ...                        
6366    Iya miris banget. Smua gara2 pengaruh barat ni...
6367    Kk luna, harus kuat ya! Harus sabar ngadepin s...
6368    Mas. Sebenarnya aku gamau bully mas tp kek nya...
6369    Kdang ngaku nicky minaj, kdang beyonce, kdang ...
6370    Time kemas sesuatu tadi, terfikir jugak botol ...
Name: text, Length: 6371, dtype: object

In [41]:
print(f'ukuran train : {df_train.shape}')


ukuran train : (6371, 6)
ukuran test : (778, 6)


In [42]:
df_train.columns


Index(['text', 'source', 'PORNOGRAFI', 'SARA', 'RADIKALISME',
       'PENCEMARAN_NAMA_BAIK'],
      dtype='object')

In [43]:
df_train['SARA'].value_counts().sum()


6371

In [45]:
df_train[df_train['SARA'] == 1].value_counts().sum()


1057

In [46]:
labels = ['SARA', 'RADIKALISME', 'PENCEMARAN_NAMA_BAIK', 'PORNOGRAFI']


In [47]:
for i in labels:
    print(
        f'presentase data train untuk label {i} adalah {np.sum(df_train[i]) / df_train.value_counts().sum():,.2f}%')


presentase data train untuk label SARA adalah 0.17%
presentase data train untuk label RADIKALISME adalah 0.17%
presentase data train untuk label PENCEMARAN_NAMA_BAIK adalah 0.31%
presentase data train untuk label PORNOGRAFI adalah 0.20%


# Preprocessing Data

In [49]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover
list_hapus = open("list_hapus.txt", "r").read().splitlines()


In [50]:
def hapus_baris(text):
    return re.sub('\n', ' ',text)

kata = 'buat apa \ntuh\n'
print(hapus_baris(kata))


buat apa  tuh 


In [51]:
def hapus_backslash(text):
    a = re.sub(r'\\+', '/', text)
    a = re.sub('/n', '\n', a)
    return a


hapus_backslash('bikin anak mulu lu nof \\nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n')


'bikin anak mulu lu nof \nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n'

In [52]:
a = hapus_backslash(
    'bikin anak mulu lu nof \\nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n')
a = hapus_baris(a)
a

'bikin anak mulu lu nof  kaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r '

In [53]:
def hapus_quote(text):
    text = re.sub('\[', ' [', text)
    text = re.sub('\]', '] ', text)
    text = re.sub('\[quote[^ ]*\].*?\[\/quote\]', ' ', text)
    text = re.sub('\[[^ ]*\]', ' ', text)
    text = re.sub('&quot;', ' ', text)
    return text


text = '[QUOTE=jessepinkman16;5a50ac34d89b093f368b456e]yoiii cuy halo halo bandung[/QUOTE]'

print('Before :', text)
print('After  :', hapus_quote(text))


Before : [QUOTE=jessepinkman16;5a50ac34d89b093f368b456e]yoiii cuy halo halo bandung[/QUOTE]
After  :    yoiii cuy halo halo bandung   


In [54]:
def hapus_link(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)


text = 'kemaren kacao bet de www.instagram.com/dj213987qekjhqew'

print('Before :', text)
print('After  :', hapus_link(text))


Before : kemaren kacao bet de www.instagram.com/dj213987qekjhqew
After  : kemaren kacao bet de 


In [55]:
def hapus_spasi_banyak(text):
    return re.sub('  +', ' ', text)


text = 'budi      pergi ke           pasar'

print('Before :', text)
print('After  :', hapus_spasi_banyak(text))


Before : budi      pergi ke           pasar
After  : budi pergi ke pasar


In [57]:
slang = pd.read_csv('data/slangword.csv')
slang


,original,translated
0,7an,tujuan
1,@,di
2,ababil,abg labil
3,abis,habis
4,acc,accord
...,...,...
2873,tauge,taoge
2874,toge,taoge
2875,taubat,tobat
2876,trilyun,triliun


In [58]:
slang_dict = dict(zip(slang['original'], slang['translated']))


In [59]:
def transform_slang_words(text):
    word_list = text.split()
    word_list_len = len(word_list)
    transformed_word_list = []
    i = 0
    while i < word_list_len:
        if (i + 1) < word_list_len:
            two_words = ' '.join(word_list[i:i+2])
            if two_words in slang_dict:
                transformed_word_list.append(slang_dict[two_words])
                i += 2
                continue
        transformed_word_list.append(
            slang_dict.get(word_list[i], word_list[i]))
        i += 1
    return ' '.join(transformed_word_list)


text = 'dikit lagi abis nih'

print('Before :', text)
print('After  :', transform_slang_words(text))


Before : dikit lagi abis nih
After  : dikit lagi habis ini


In [60]:
def remove_non_alphabet(text):
    output = re.sub('[^a-zA-Z ]+', '', text)
    return output


text = 'kemaren tu123 ada kelinci di kebun.'

print('Before :', text)
print('After  :', remove_non_alphabet(text))


Before : kemaren tu123 ada kelinci di kebun.
After  : kemaren tu ada kelinci di kebun


In [61]:
def remove_twitter_ig_formatting(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'\brt\b', '', text)
    return text


text = '@jonijon menurut saya hal tersebut masih kurang baik dilakukan sih kak'

print('Before :', text)
print('After  :', remove_twitter_ig_formatting(text))


Before : @jonijon menurut saya hal tersebut masih kurang baik dilakukan sih kak
After  :  menurut saya hal tersebut masih kurang baik dilakukan sih kak


In [62]:
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
print(stopwords)


['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [63]:
removeFactory = StopWordRemoverFactory()
stopwords = removeFactory.create_stop_word_remover()
stopwords_plus = removeFactory.get_stop_words()+list_hapus
dictionary = ArrayDictionary(stopwords_plus)

stopwords = StopWordRemover(dictionary)

def removeStopWords(text):
    stop = stopwords.remove(text)
    return stop


text = 'Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah.'
print('Before :', text)
print('After  :', removeStopWords(text))


Before : Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah.
After  : Andi kerap melakukan transaksi rutin daring online. Menurut Andi belanja online lebih praktis & murah.


In [64]:
stemFactory = StemmerFactory()
stemmer = stemFactory.create_stemmer()


def stemming(text):
    hasil = stemmer.stem(text)
    return hasil


text = "Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah."

print('Before :', text)
print('After  :', stemming(text))


Before : Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah.
After  : andi kerap laku transaksi rutin cara daring atau online turut andi belanja online lebih praktis murah


In [65]:
def remove_repeating_characters(text):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(text))


text = 'heyyyyyyyyyyyyyyyyyyyy kenapa tadi?'

print('Before :', text)
print('After  :', remove_repeating_characters(text))


Before : heyyyyyyyyyyyyyyyyyyyy kenapa tadi?
After  : hey kenapa tadi?


In [66]:
def preprocessing(text):
    text = text.lower()
    text = hapus_quote(text)
    text = hapus_backslash(text)
    text = hapus_baris(text)
    text = hapus_link(text)
    text = remove_twitter_ig_formatting(text)
    text = remove_repeating_characters(text)
    text = transform_slang_words(text)
    text = remove_non_alphabet(text)
    text = hapus_spasi_banyak(text)
    text = removeStopWords(text)
    text = stemming(text)
    text = text.lower()
    return text


In [67]:
import time

In [68]:
%%time
start = time.time()

df_train['text'] = df_train['text'].apply(preprocessing)
df_train
end = time.time()
print(f'Lama waktu run dengan Stemming : {end-start:,.5f} detik')


Lama waktu run dengan Stemming : 1,348.72069 detik
CPU times: total: 22min 25s
Wall time: 22min 28s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [82]:
def preprocessing_tanpa_stemming(text):
    text = text.lower()
    text = hapus_quote(text)
    text = hapus_backslash(text)
    text = hapus_baris(text)
    text = hapus_link(text)
    text = remove_twitter_ig_formatting(text)
    text = remove_repeating_characters(text)
    text = transform_slang_words(text)
    text = remove_non_alphabet(text)
    text = hapus_spasi_banyak(text)
    text = removeStopWords(text)
    # text = stemming(text)
    text = text.lower()
    return text


In [85]:
%%time
start = time.time()

df_train['text'] = df_train['text'].apply(preprocessing_tanpa_stemming)
df_train
end = time.time()
print(f'Lama waktu run tanpa Stemming : {end-start:,.5f} detik')


Lama waktu run tanpa Stemming : 2.36368 detik
CPU times: total: 2.36 s
Wall time: 2.36 s


In [69]:
df_train['text'][6301]


'kak cantik anget aku bilang aku bobo tetanga belah sedang apa mandin sih meong sama punya rumah lah'

In [70]:
df_train


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,woi anjing bodoh provinsi paling banyak ngerus...,kaskus,0,0,0,1
1,saja nitizen penasaran keluarga besar sudah pa...,instagram,0,0,0,0
2,sidangahok moga sipenista agama ateknya mati w...,twitter,0,1,1,1
3,jakarta barusan baca undang tetap beda kok pre...,instagram,0,0,0,0
4,buat anak melulu kamu nof mikir apa kasi anak ...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6366,iya miris sekali semua gara pengaruh barat sep...,kaskus,0,0,0,0
6367,luna kuat iya sabar ngadepin sikap benci mungk...,instagram,0,0,0,0
6368,mas benar aku mau buly mas seperti nya mas min...,instagram,0,0,0,0
6369,kdang aku nicky minaj kdang beyonce kdang arti...,instagram,0,0,0,0


In [71]:
# df.to_csv('data fix/data_kalimat.csv', index=False)


In [72]:
df_train['text']


0       woi anjing bodoh provinsi paling banyak ngerus...
1       saja nitizen penasaran keluarga besar sudah pa...
2       sidangahok moga sipenista agama ateknya mati w...
3       jakarta barusan baca undang tetap beda kok pre...
4       buat anak melulu kamu nof mikir apa kasi anak ...
                              ...                        
6366    iya miris sekali semua gara pengaruh barat sep...
6367    luna kuat iya sabar ngadepin sikap benci mungk...
6368    mas benar aku mau buly mas seperti nya mas min...
6369    kdang aku nicky minaj kdang beyonce kdang arti...
6370    waktu kemas tadi terfikir jugak botol air keci...
Name: text, Length: 6371, dtype: object

In [73]:
# df_train = df_train


In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [75]:
vectorizer = TfidfVectorizer()



In [76]:
result = vectorizer.fit_transform(df_train['text']).toarray()
result


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [77]:
df_baru = pd.DataFrame(result, columns=vectorizer.get_feature_names_out())
df_baru


,ab,abad,abadi,abah,abai,abang,abangan,abas,abasiah,abasid,...,zona,zoroaster,zouma,zoya,zuhur,zulkarnaen,zulkarnain,zulkifli,zy,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df = pd.concat([df_train[['PORNOGRAFI', 'SARA', 'RADIKALISME',
               'PENCEMARAN_NAMA_BAIK']], df_baru], axis=1)
df


,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK,ab,abad,abadi,abah,abai,abang,...,zona,zoroaster,zouma,zoya,zuhur,zulkarnaen,zulkarnain,zulkifli,zy,zynga
0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6366,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6367,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6368,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6369,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
# df.to_csv('data fix/preprocess_fix_backslash_wo_stemming.csv', index=False)
